# Async for the Python 2 Programmer
<center><img src="caveman_lawyer.jpg"></center>
## Henry Chen (Hackbright Academy)

# Goals
- implement with familiar **Python 2** syntax
- connect to new **Python 3** syntax
- **NOT** a tutorial on writing async code
- prepare you for **further study**
- develop **mental model** of async

# Concurrency

- programs often wait around
- http requests, file read/write, etc
- concurrency: program does stuff while waiting
- more efficient use of CPU

# async vs multi-threading
async
- one sequence of computations
- how programs normally work
- you control order of execution

multi-threading
- parallel sequences
- OS decides what to do next
- but you can add constraints

# You probably already know async

In JavaScript,

```javascript
setTimeout(function(){alert("Hello")}, 1000);
```

raises alert after delay of 1000 ms.

- setTimeout adds a function to a task list
- event loop continuously "checks" each task in turn
- when delay expires, executes callback

Let's implement this in Python...

In [ ]:
from datetime import datetime, timedelta

def sleep(delay):
    expire_at = datetime.now() + timedelta(seconds=delay)
    while datetime.now() < expire_at:
        pass
    
def set_timeout(delay, callback):
    sleep(delay) # wait for a duration
    callback() # execute callback function

In [ ]:
def alert(msg):
    print(msg)

set_timeout(1.0, lambda: alert('goodbye')) # lambda needed to defer execution
set_timeout(0.5, lambda: alert('hello'))

🤮🤮🤮🤮🤮🤮🤮🤮🤮

# try again, use generators
- can pause and resume
- loop thru several generators
- execute each a little at a time


In [ ]:
def f():
    yield 42
    yield False
    yield 'hello'

In [ ]:
it = f()
it

In [ ]:
it.send(None)

In [ ]:
it = f()
for item in it:
    print(item)

In [ ]:
# replace normal functions with generator functions

def sleep(delay):
    expire_at = datetime.now() + timedelta(seconds=delay)
    
    while datetime.now() < expire_at:
        yield None
        
def set_timeout(delay, callback):
    for item in sleep(delay):
        yield item
    callback()

In [ ]:
gen = set_timeout(2, lambda: alert('hi'))
gen

In [ ]:
gen.send(None) # returns None until delay expires

Unlike JavaScript, Python does not come with an event loop, so let's make one.

In [ ]:
def run(tasks):
    
    while tasks:
        task = tasks.pop(0)
        
        try:
            task.send(None)
        except StopIteration:
            pass
        else:
            tasks.append(task)

In [ ]:
run([
    set_timeout(1, lambda: alert('goodbye')),
    set_timeout(0.5, lambda: alert('hello'))
    ])

😎😎😎😎😎😎😎😎😎😎😎😎

# Python 3

- Every line of code so far can be run under **Python 2** (try it!)
- Python 3 introduces new **syntax** and standard library features specifically for async IO
- But under it all is the good old **generator**

In [ ]:
import asyncio # Python 3 module with non-blocking IO and other utilities

In [ ]:
@asyncio.coroutine # tells Python this is a coroutine, not a normal generator function
def sleep(delay):
    expire_at = datetime.now() + timedelta(seconds=delay)
    while datetime.now() < expire_at:
        yield
        

async def set_timeout(delay, callback): # Python 3 syntax
    await sleep(delay) # exhaust the coroutine
    callback()

In [ ]:
# our event loop still works!
run([
    set_timeout(1, lambda: alert('goodbye')),
    set_timeout(0.5, lambda: alert('hello'))
    ])

In [ ]:
# or, use the official event loop
loop = asyncio.get_event_loop()

In [ ]:
tasks = asyncio.gather(
    set_timeout(1, lambda: alert('goodbye')),
    set_timeout(0.5, lambda: alert('hello'))
)

In [ ]:
loop.run_until_complete(tasks)

# Summary

- async/await and asyncio are decoupled
    - you can use one or both or neither
- ideas and machinery already present in Python 2
    - but Python 3 is nicer
- single thread is simpler than multiple threads
    - but your entire code must be non-blocking

# Further Study

- DO NOT read the docs
- watch David Beazley's async/await talks on YouTube
- bonus tip: watch Beazley talk about basically anything
- read [How the heck does async/await work in Python 3.5?](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/) by Brett Cannon

Thanks for your attention!

henry@hackbrightacademy.com

github.com/scotchka/async-py2-talk